In [121]:
import requests
import pandas as pd
import tensorflow as tf
import base64
import json


In [122]:
data = pd.read_csv("./data/Obesity Classification.csv")


In [123]:
data.drop(columns=["ID"], axis=1).to_csv("data/data.csv")

In [124]:
data

,ID,Age,Gender,Height,Weight,BMI,Label
0,1,25,Male,175,80,25.3,Normal Weight
1,2,30,Female,160,60,22.5,Normal Weight
2,3,35,Male,180,90,27.3,Overweight
3,4,40,Female,150,50,20.0,Underweight
4,5,45,Male,190,100,31.2,Obese
...,...,...,...,...,...,...,...
103,106,11,Male,175,10,3.9,Underweight
104,107,16,Female,160,10,3.9,Underweight
105,108,21,Male,180,15,5.6,Underweight
106,109,26,Female,150,15,5.6,Underweight


In [125]:
data["Label"].unique()

array(['Normal Weight', 'Overweight', 'Underweight', 'Obese'],
      dtype=object)

In [126]:
print(requests.get("https://obesityclassification-production.up.railway.app/v1/models/cc-model").json())


{'model_version_status': [{'version': '1751720916', 'state': 'AVAILABLE', 'status': {'error_code': 'OK', 'error_message': ''}}]}


In [ ]:

def prepare_json(inputs: dict):

    # Mendefinisikan mapping dari kunci input ke jenis Fitur TensorFlow
    feature_mapping = {
    "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Age'])])),
    "Gender": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['Gender'].encode()])),
    "BMI": tf.train.Feature(float_list=tf.train.FloatList(value=[float(inputs['BMI'])])),
    "Height": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Height'])])),
    "Weight": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Weight'])])),
    }


    # Membuat TensorFlow Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Melakukan encode Example ke base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    # Mengembalikan data dalam format JSON
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })


# Fungsi untuk memprediksi data
def predict(inputs):
    json_data = prepare_json(inputs)
    endpoint = "https://obesityclassification-production.up.railway.app/v1/models/cc-model:predict"

    try:
        response = requests.post(endpoint, data=json_data, timeout=10) 
        response.raise_for_status() 
        print("Response JSON:", response.json())
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Response content: {response.text}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An unexpected error occurred: {req_err}")

    return None




In [128]:
datas = data.drop(columns=["Label"], axis=1)

In [129]:
oke = predict(datas.iloc[0])
oke

Response JSON: {'predictions': [[0.0, 0.992513955, 0.00748599553, 6.13071634e-26]]}


{'predictions': [[0.0, 0.992513955, 0.00748599553, 6.13071634e-26]]}